## Your Model 🌱Garden🌱 Execution Environment

Use this notebook to write a function that executes your model(s). Tag that function with the `@garden_pipeline` decorator.

Garden will take this notebook and build a container with it. When Garden executes your `@garden_pipeline`, it will be like like you have just run all the cells of this notebook once. So you can install libraries with `!pip install` and your function can use those libraries. You can also define helper functions and constants to use in your `@garden_pipeline`.

In [ ]:
!pip install garden-ai
!pip install tensorflow=="2.14.0"

In [1]:
from garden_ai.model_connectors import HFConnector
from garden_ai import PipelineMetadata, garden_pipeline

import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist

### Model connectors

Model connectors let Garden import metadata about your model.
They also have a `stage` method that you can use to download your model weights.

In [3]:
my_hugging_face_repo = HFConnector("willengler-uc/keras-mnist")

### Pipeline metadata


To publish your function, Garden needs metadata so that other users can discover it.
Edit this PipelineMetadata object to describe your function.


In [9]:
my_pipeline_meta = PipelineMetadata(
    title="Keras MNIST digit classifier",
    description="Classify fuzzy black and white digits with TensorFlow",
    authors=["Will Engler"],
    tags=["MNIST digits", "Keras"]
)

### Write your pipeline function that will run remotely

The `@garden_pipeline` decorator makes this function available to run in your garden when you publish the notebook.
Download your model weights and call your model in this function.

In the decorator be sure to include:
- your pipeline metadata,
- connectors for any models you're using,
- the DOI of the garden you want this pipeline to be found in. (Check `garden-ai garden list` for the DOIs of your gardens.)

In [14]:
@garden_pipeline(metadata=my_pipeline_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.23677/j4cg-6427")
def classify_digit(image):
    download_path = my_hugging_face_repo.stage()
    model = load_model(f"{download_path}/mnist_model.keras")
    prediction = model.predict(image)
    return np.argmax(prediction)

### Test your pipeline function

Finally, make sure your `@garden_pipeline` works!
When Garden makes a container from your notebook, it runs all the cells in order and saves the notebook. Then users invoke your `@garden_pipeline` in the context of the notebook.

If you can hit "Kernel" -> "Restart and run all cells" and your test below works, your `@garden_pipeline` will work in your garden!


In [15]:
def test_the_model():
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
    sample_image = test_images[0]
    sample_image = np.expand_dims(sample_image, axis=0)
    return classify_digit(sample_image)

In [ ]:
#test_the_model()